In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps
import seaborn as sns

Коэффициент корелляции Пирсона двух выборок оценивается по формуле
$$ \hat\theta = 
\frac{
\sum\limits_{i=1}^n(X_i - \overline{X})(Y_i - \overline{Y})
}{
\sqrt{\sum\limits_{i=1}^n(X_i - \overline{X})^2
\sum\limits_{j=1}^n(Y_j - \overline{Y})^2
}
}
$$ 
Он служит приближением для, собственно, корелляции двух выборок. В некотором приближении справедлива следующая формула:
$$arth \theta | (X, Y) \sim N(arth \hat{\theta}, 1 / n)$$
Здесь $arth(x) = \frac{1}{2}ln\frac{1+x}{1-x}$ - монотонная функция

In [0]:
arth = lambda x: 0.5 * np.log(1 + x) / np.log(1 - x)

class HypotheseCorellation:
# Нужно объединить с другим классом, для случая 
  def __init__(self):
    pass

  def test(self, X, Y, test_params):
    """
    :param X: - первая выборка
    :param Y: - вторая выборка
    :param  test_params - словарь с параметрами гипотезы
    обязательно содержит:
    "type_of_test" - одно из "modification", "lindley", "leq", "geq" - тип гипотезы 
    "t0": собственно, точка, относительно которой проверяется гипотеза
    для метода модификации гипотезы: "epsilon" - радиус "окрестности" t0
    для метода Линдли - "conf" - уроветь зачимости
    возвращает метод также словарь, в зависимости от гипотезы может 
    содержать "HDR" в виде концов отрезка - важно: именно для
    arth theta, а не theta (за исключением метода модификации критерия), "rejected" - для метода Линдли,
    "p0", "p1", "B" - апостериорные вероятности и байесовский фактор,
    "post_params" - ппараметры постериорного распределения, см. выкладки и примеры
    применения
    """
    t0 = test_params["t0"]
    n = len(X)
    test_type = test_params["type_of_test"]

    assert test_type in ["modification", "lindley", "leq", "geq"], "unknown test type"

    corr_coef = np.corrcoef(X, Y)[1][0]
    t0_arth = arth(t0)
    distr = sps.norm(arth(corr_coef), np.sqrt(1/n)) # апостериорное распредение корелляции
     
    if test_type == "lindley":
        try:
          conf = 1 - test_params["conf"]
        except Exception:
          print("not enough params")
        l, r = distr.ppf((1 - conf)/2), distr.ppf((1 + conf)/2)
        reject = False
        if t0_arth < l or t0_arth > r:
          reject = True
        return {
            "reject":reject,
            "HDR": (l, r),
            "post_params": {
                "mean": arth(corr_coef),
                "sigma^2": 1/n
            } 
        }

    if test_type == "modification":
          try:
            epsilon = test_params["epsilon"]
          except Exception:
            print("not enough params")
          p0 = distr.cdf(arth(t0 + epsilon)) - distr.cdf(arth(t0 - epsilon))

          # под HDR в данном случае подразумеваем такую симметричную окрестность  
          # среднего, что t0 в неё попадает "а границе"

          #Возможно, есть смысл сделать по другому?
          delta = np.abs(corr_coef - t0)
          l, r = (corr_coef- delta, corr_coef + delta)
          return {
              "p0": p0,
              "p1": 1 - p0,
              "HDR": (l, r),
              "post_params": {
                  "mean": arth(corr_coef),
                  "sigma^2": 1/n
              } 
          }
    
    if test_type in ["leq", "geq"]:
      p0 = distr.cdf(t0_arth)
      p1 = 1 - p0
      if test_type == "geq":
        p0, p1 = p1, p0
      B = p0 / p1
      # При чем тут HDR, не очень понял
      return {
          "p0": p0,
          "p1": p1, 
          "B": B
      }

In [0]:
def generate_sample(corr=0.5, n = 100):
  m1 = 1
  m2 = 2
  d1 = 1
  d2 = 2
  cov = corr * np.sqrt(d1 * d2)
  cov_mat = np.array([
      [d1, cov],
      [cov, d2]]
  )
  X, Y = sps.multivariate_normal(mean=np.array([m1, m1]), cov=cov_mat).rvs(n).T
  return X, Y

##Примеры применения.
Один класс может тестировать различные типы гипотез, в том числе и "сложная против сложной"

In [0]:
hypo = HypotheseCorellation()

In [0]:
X, Y = generate_sample()

In [139]:
test_type_2 = {
    "type_of_test": "modification",
    "t0": 0.5,
    "epsilon": 0.1,
}
hypo.test(X, Y, test_type_2)

{'HDR': (0.35854466449975486, 0.5),
 'p0': 0.2756893275304756,
 'p1': 0.7243106724695244,
 'post_params': {'mean': -0.31841832593640074, 'sigma^2': 0.01}}

In [141]:
test_type_3 = {
    "type_of_test": "lindley",
    "t0": 0.5,
    "conf": 0.05
}
hypo.test(X, Y, test_type_3)

{'HDR': (-0.5144147243904061, -0.12242192748239533),
 'post_params': {'mean': -0.31841832593640074, 'sigma^2': 0.01},
 'reject': False}

In [143]:
test_type_4 = {
    "type_of_test": "leq",
    "t0": 0.7
}
hypo.test(X, Y, test_type_4)

{'B': 5.119470566293664, 'p0': 0.836587170545758, 'p1': 0.16341282945424196}

In [146]:
test_type_5 = {
    "type_of_test": "geq",
    "t0": 0.7
}
hypo.test(X, Y, test_type_5)

{'B': 0.19533269838173298, 'p0': 0.16341282945424196, 'p1': 0.836587170545758}

К сожалению, области HDR теперь не очень информативны.

Посмотрим, что функции верно работают

In [150]:
test_type = {
    "type_of_test": "leq",
    "t0": 0.4
}
for n in [100, 100, 10000]:
  print("n: ", n)
  X, Y = generate_sample(n=n)
  print(hypo.test(X, Y, test_type))

n:  100
{'p0': 0.5778749145361427, 'p1': 0.4221250854638573, 'B': 1.3689660587243655}
n:  100
{'p0': 0.2891312830443852, 'p1': 0.7108687169556148, 'B': 0.40672950735914565}
n:  10000
{'p0': 0.00023727939802313888, 'p1': 0.9997627206019769, 'B': 0.00023733571289822475}


Мы все увереннее отвергаем гипотезу с ростом  n

In [166]:
test_type_3 = {
    "type_of_test": "lindley",
    "t0": 0.5,
    "conf": 0.05
}
for n in [10, 100, 10000]:
  print("n: ", n)
  X, Y = generate_sample(n=n)
  print(hypo.test(X, Y, test_type_3))

n:  10
{'reject': False, 'HDR': (-0.9697617980101267, 0.26982826659899634), 'post_params': {'mean': -0.34996676570556523, 'sigma^2': 0.1}}
n:  100
{'reject': False, 'HDR': (-0.4820940978642314, -0.0901013009562206), 'post_params': {'mean': -0.286097699410226, 'sigma^2': 0.01}}
n:  10000
{'reject': False, 'HDR': (-0.3162934467012189, -0.27709416701041784), 'post_params': {'mean': -0.29669380685581836, 'sigma^2': 0.0001}}


In [167]:
test_type_3 = {
    "type_of_test": "lindley",
    "t0": 0.4,
    "conf": 0.05
}
for n in [10, 100, 10000]:
  print("n: ", n)
  X, Y = generate_sample(n=n)
  print(hypo.test(X, Y, test_type_3))

n:  10
{'reject': False, 'HDR': (-0.8916170502513098, 0.34797301435781336), 'post_params': {'mean': -0.2718220179467482, 'sigma^2': 0.1}}
n:  100
{'reject': False, 'HDR': (-0.4816371473359856, -0.0896443504279748), 'post_params': {'mean': -0.2856407488819802, 'sigma^2': 0.01}}
n:  10000
{'reject': True, 'HDR': (-0.3092565167976778, -0.27005723710687674), 'post_params': {'mean': -0.28965687695227726, 'sigma^2': 0.0001}}


Метод Линдли на практике показал себя не очень мощеным, но работает